### Exercise 1.6: 
Alyssa P. Hacker doesn’t see why $if$ needs to be provided as a special form. “Why can’t I just define it as an ordinary procedure in terms of $cond$?” she asks. Alyssa’s friend Eva Lu Ator claims this can indeed be done, and she defines a new version of $if$:

Eva demonstrates the program for Alyssa:

Delighted, Alyssa uses $new$-$if$ to rewrite the $square$-$root$ program:

What happens when Alyssa attempts to use this to compute square roots? Explain.

### 解答：

使用new-if过程来构建新的平方根过程：

In [1]:
cat square-root-by-new-if.scm

(define (average x y) (/ (+ x y) 2))
(define (improve guess x) (average guess (/ x guess)))

(define (good-enough? guess x)
  (< (abs (- (square guess) x)) 0.001))

;;The define of new-if
(define (new-if predicate then-clause else-clause)
  (cond (predicate then-clause) (else else-clause)))

;;The define of sqrt-iter used by new-if
(define (sqrt-iter guess x)
  (new-if (good-enough? guess x)
          guess
          (sqrt-iter (improve guess x) x)))

(define (square-root x) (sqrt-iter 1.0 x))


#### Running Instance:

1 ]=> (load "square-root-by-new-if.scm")

;Loading "square-root-by-new-if.scm"... done
;Value: square-root

1 ]=> (square-root 9)

;Aborting!: maximum recursion depth exceeded

#### 原因分析:

The special form $if$, a restricted type of conditional that can be used when there are precisely two cases in the case analysis.

To evaluate an $if$ expression, the interpreter starts by evaluating the ⟨predicate⟩ part of the expression. If the ⟨predicate⟩ evaluates to a true value, the interpreter then evaluates the ⟨consequent⟩ and returns its value. Otherwise it evaluates the ⟨alternative⟩ and returns its value.

另一方面，new-if只是一个普通过程，没有if所具有的特殊形式，根据解释器使用的应用序求值规则，当过程调用时，该过程的实际参数在传入的时候都会被求值。因此当使用new-if时，无论predicate的真假，then-clause和else-clause都会被求值。

起初让我困惑的地方在于，混淆了条件表达式cond的求值规则和一般过程应用序求值规则之间的关系。在上面的new-if的测试实例中

在实际参数(= 2 3),0和5传入过程new-if的过程体中时，已经按照应用序求值规则对它们进行了求值，分别的求值结果为#f，0和5，然后按照cond条件表达式的求值规则返回对应的值。这里需要特别注意的是：在cond条件表达式进行计算之前，其谓语和所有对应的分支值已经被计算出来了，cond所做的工作仅仅是根据#t或者#f来返回第一个或者第二个值。

为了验证这一理解是否合理，我们使用display过程构造一个new-if的调用，对过程display的调用会立即输出相应的内容：

我们对第一个调用进行分析，按照上面的理解，首先按照应用序求值规则对谓语(> 2 3)和then分支语句(display "then clause")即else分支语句(+ 2 3)进行求值，分别得到#f,在屏幕上输出"then clause"，以及整数5；然后进入new-if过程体中cond表达式进行求值，由于谓语为#f，则cond表达式返回else分支语句的值，即5，所以有如上的运行结果。对于第二个调用来说，在执行cond表达式求值之前，得到的应用序求值结果分别是#t，在屏幕上输出"then clause",以及整数5，在执行cond表达式求值时，由于谓语为#t，则返回then分支语句的值作为cond表达式的返回值，然而display过程在上面传入前被调用后无返回值，所以会在屏幕输出"Unspecified return value"的提示，所以有如上的运行结果。

通过上面这个实例，我们就不难理解在习题集中给出的另外一个实例：

#### 总结：

根据上述原因分析，说明了为什么用new-if重新定义的sqrt-iter会导致错误：  
无论good-enough?过程的测试结果如何，sqrt-iter过程会一直递归下去，直到栈溢出。